In [16]:
import pandas as pd
import os

In [17]:
base_path = 'D:\Research\Data\QueryReformulaiton\Forgotten_rule_query_reformulation-master\\'
ground_truth_path = base_path + '\GroundTruth'
reformulated_query_path = base_path + 'NrOptimal-GA\Query'
bug_report_path = base_path + '\BugReports\ALL'

In [18]:
# read folder names from bug report folder
bug_report_folders = os.listdir(bug_report_path)
# repositories =

In [19]:
bug_repos = bug_report_folders

In [20]:
# from ground truth, based on bug repos, read the ground truth. for each bug repo, there are multiple ground truth files
# for each repo create a dataframe with columns: bug_id, which is the file name, and the file associated with it as ground truth




## Step: 1 first dataframe with bug_id and ground truth

In [21]:
# iterate over bug_repos and read the ground truth files
ground_truth_df = pd.DataFrame(columns=['bug_id', 'ground_truth', 'repo'])
for repo in bug_repos:
    cur_repo_path = ground_truth_path + '\\' + repo
    ground_truth_files = os.listdir(cur_repo_path)
    for file in ground_truth_files:
        cur_file_path = cur_repo_path + '\\' + file
        bug_id, _ = os.path.splitext(file)
        # read the content of the file as a string as ground truth
        with open(cur_file_path, 'r') as f:
            # one bug can have multiple ground truths. each line is a ground truth. read line wise add to a list
            ground_truth = f.readlines()
            # remove the new line character from the end of each line
            ground_truth = [x.strip() for x in ground_truth]


        # Creating a new DataFrame for the new row
        new_row = pd.DataFrame({'bug_id': [bug_id], 'ground_truth': [ground_truth], 'repo': [repo]})

        # Concatenate the new row DataFrame with the existing DataFrame
        ground_truth_df = pd.concat([ground_truth_df, new_row], ignore_index=True)

In [22]:
ground_truth_df.head()

,bug_id,ground_truth,repo
0,112599,[providers/bundles/org.eclipse.ecf.provider.xm...,ecf
1,125572,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf
2,134483,[framework/bundles/org.eclipse.ecf/src/org/ecl...,ecf
3,146622,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf
4,147269,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf


## Step-2: Second dataframe with bug_id and reformulated query

In [23]:
# all_reformed_query_files = os.listdir(reformulated_query_path)
# dict_bug_id_query = {}

which_file = 'GA-ALL-QE-K10-1.txt'
reformed_query_df = pd.DataFrame(columns=['bug_id', 'reformed_query', 'repo'])
for repo in bug_repos:

    all_reformed_query_files = os.listdir(os.path.join(reformulated_query_path, repo))
    dict_bug_id_query = {}

    # now, iterate through all the reformed query file
    for reformed_query_file in all_reformed_query_files:
        which_file = reformed_query_file
        cur_query_file = reformulated_query_path + '\\' + repo + '\\' + which_file
        # read the file line by line. each line first few characters are the bug id separated by a tab; then the query
        with open(cur_query_file, 'r') as f:
            for line in f:
                bug_id = line.split('\t')[0]
                query = line.split('\t')[1]

                # remove the new line character from the end of each line
                query = query.strip()

                # check if bug_id exists in the dictionary
                if bug_id not in dict_bug_id_query:
                    reformed_query_array = [query]
                    dict_bug_id_query[bug_id] = reformed_query_array
                else:
                    reformed_query_array = dict_bug_id_query[bug_id]
                    reformed_query_array.append(query)
                    dict_bug_id_query[bug_id] = reformed_query_array

    # now, iterate through the dictionary
    for key, value in dict_bug_id_query.items():
        # Creating a new DataFrame for the new row
        new_row = pd.DataFrame({'bug_id': [key], 'reformed_query': [value], 'repo': [repo]})

        # Concatenate the new row DataFrame with the existing DataFrame
        reformed_query_df = pd.concat([reformed_query_df, new_row], ignore_index=True)

In [24]:
reformed_query_df.head()

,bug_id,reformed_query,repo
0,209410,[TODOs Bug properties comments HTMLified Bug c...,ecf
1,211585,"[service parsing stumbles code type, parsing D...",ecf
2,318086,[build moment project build ecf tests org mome...,ecf
3,238976,[Editor modifications editor remote shared rem...,ecf
4,193415,[contacts Account accounts disconnect chevron ...,ecf


In [25]:
reformed_query_df['reformed_query'][0]

['TODOs Bug properties comments HTMLified Bug commands feedback improved bot',
 'handling HTMLified Bug comments properties bot file TODOs Bug documentation',
 'HTMLified HTMLified details simply Bug documentation improved commands Bug bot',
 'attachment commands fixes documentation improved Bug feedback Bug improve regexes',
 'Created attached bot details HTMLified documentation improved Created properties documentation attachment commands attached Bug patch',
 'simply patch misc TODOs Bug HTMLified simply commands Bug TODOs feedback properties improved bot misc properties Created feedback Bug attached',
 'fixes Bug feedback simply commands']

## Step-3: Third dataframe with bug_id and bug report

In [26]:
# iterate over bug_repos and read the ground truth files
bug_report_df = pd.DataFrame(columns=['bug_id', 'bug_title', 'bug_description', 'repo'])
for repo in bug_repos:
    cur_repo_path = bug_report_path + '\\' + repo
    bug_report_files = os.listdir(cur_repo_path)
    for file in bug_report_files:
        cur_file_path = cur_repo_path + '\\' + file
        bug_id, _ = os.path.splitext(file)
        # read the content of the file as a string as ground truth
        with open(cur_file_path, 'r') as f:
            bug_title = f.readline()
            # remove the new line character from the end of each line
            bug_title = bug_title.strip()

            # replace the string format 'Bug+space+bug_id' from the beginning of the title
            bug_title = bug_title.replace('Bug ' + bug_id, '')
            bug_title = bug_title.replace(bug_id, '')

            bug_description = f.read()

        # Creating a new DataFrame for the new row
        new_row = pd.DataFrame({'bug_id': [bug_id], 'bug_title': [bug_title], 'bug_description': [bug_description], 'repo': [repo]})

        # Concatenate the new row DataFrame with the existing DataFrame
        bug_report_df = pd.concat([bug_report_df, new_row], ignore_index=True)

In [27]:
bug_report_df.head()

,bug_id,bug_title,bug_description,repo
0,112599,– [XMPP] Room subject does not get updated in...,When updated remotely by xmpp server title of ...,ecf
1,125572,– ECF Generic provider thread interlock,We see the following problem while running an ...,ecf
2,134483,– Standalone ClientApplication is breaks in l...,The standalone org.eclipse.ecf.provider.app.Cl...,ecf
3,146622,– deserializeSharedObjectMessage with custom ...,when sending a instance of a custom Class in a...,ecf
4,147269,"– The ""send file"" functionality fails and lau...",>>> Environment: WinXP + Java 1.5.0_06 + Eclip...,ecf


## Step 4: Merge the three dataframes

In [28]:
# merge the three dataframes based on 'bug_id'

In [29]:
merged_df = pd.merge(ground_truth_df, reformed_query_df, on=['bug_id', 'repo'])

In [30]:
merged_df = pd.merge(merged_df, bug_report_df, on=['bug_id', 'repo'])

In [31]:
merged_df.head()

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description
0,112599,[providers/bundles/org.eclipse.ecf.provider.xm...,ecf,[updated xmpp xmpp chat updated updated room x...,– [XMPP] Room subject does not get updated in...,When updated remotely by xmpp server title of ...
1,125572,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf,[Suspended Context client Connection Container...,– ECF Generic provider thread interlock,We see the following problem while running an ...
2,134483,[framework/bundles/org.eclipse.ecf/src/org/ecl...,ecf,[Application Container Standalone Factory Cont...,– Standalone ClientApplication is breaks in l...,The standalone org.eclipse.ecf.provider.app.Cl...
3,146622,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf,[handleAsynchEvent bins handleSharedObjectMess...,– deserializeSharedObjectMessage with custom ...,when sending a instance of a custom Class in a...
4,147269,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf,[group Group Thread SOManager crash load share...,"– The ""send file"" functionality fails and lau...",>>> Environment: WinXP + Java 1.5.0_06 + Eclip...


### Check stats

In [32]:
merged_df.shape

(2320, 6)

In [33]:
len(merged_df['bug_id'].unique())


2318

In [34]:
merged_df[merged_df.duplicated(['bug_id', 'repo'])]

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description


## Step 5: Save the merged dataframe as a json file

In [49]:
# convert each row of the dataframe to a json object and save it in a json file


In [36]:
import json


In [37]:
# convert the dataframe to a list of dictionaries
merged_df_dict = merged_df.to_dict('records')

In [38]:
# save the list of dictionaries as a json file
with open('../../Data/Augmented/All_Data.json', 'w') as f:
    json.dump(merged_df_dict, f)

#### read the json file and convert it to a dataframe

In [39]:
import json

file_path = '../../Data/Augmented/All_Data.json'

try:
    with open(file_path, 'r') as f:
        file_contents = f.read()
        # print(file_contents)  # Print the contents of the file

    data = json.loads(file_contents)
    # Process the JSON data here

except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
except FileNotFoundError:
    print(f"File not found: '{file_path}'")
except Exception as e:
    print("Error:", e)

#### convert the list of dictionaries to a dataframe

In [40]:
# convert the list of dictionaries to a dataframe
df = pd.DataFrame.from_dict(data)

In [41]:
df.head()

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description
0,112599,[providers/bundles/org.eclipse.ecf.provider.xm...,ecf,[updated xmpp xmpp chat updated updated room x...,– [XMPP] Room subject does not get updated in...,When updated remotely by xmpp server title of ...
1,125572,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf,[Suspended Context client Connection Container...,– ECF Generic provider thread interlock,We see the following problem while running an ...
2,134483,[framework/bundles/org.eclipse.ecf/src/org/ecl...,ecf,[Application Container Standalone Factory Cont...,– Standalone ClientApplication is breaks in l...,The standalone org.eclipse.ecf.provider.app.Cl...
3,146622,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf,[handleAsynchEvent bins handleSharedObjectMess...,– deserializeSharedObjectMessage with custom ...,when sending a instance of a custom Class in a...
4,147269,[framework/bundles/org.eclipse.ecf.provider/sr...,ecf,[group Group Thread SOManager crash load share...,"– The ""send file"" functionality fails and lau...",>>> Environment: WinXP + Java 1.5.0_06 + Eclip...


In [56]:
# df.shape

(2320, 6)

In [58]:
# # check if there are any duplicates in the bug_id column
# len(df['bug_id'].unique())
#

2318

In [60]:
# df[df.duplicated(['bug_id', 'repo'])]

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description


Some repo has same bug id. So, merging based on bug_id and repo worked.